# 1 Data Exploration
    a. Explore the dataset by displaying the first few rows, summary statistics, and data types of each column.
    b. Identify missing values, outliers, and unique values in categorical columns.

## 1.1 store sales
### 1.1.a Explore the dataset



In [ ]:
from pathlib import Path
from matplotlib.colors import ListedColormap, BoundaryNorm
import seaborn as sns
import matplotlib.ticker as mtick
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
from qtconsole.mainwindow import background
from scipy import stats

DATA_DIR = Path("data/assigment_1/store-sales-item-time-series")

stores = pd.read_csv(DATA_DIR / "stores.csv",
                          names=["store_nbr", "city", "state", "type", "cluster" ],
                          header=0)

#### Oil

In [ ]:
# 1. Overview


oil_df = pd.read_csv(DATA_DIR / "oil.csv", names=["date", "oil_price"], header=0, parse_dates=["date"])

print("=== Head: ===")
print(oil_df.head())

print("=== Info: ===")
print(oil_df.info())

print("=== Description: ===")
print(oil_df.describe())

In [ ]:
print("=== Time Period: ===")
print("Begin:", oil_df["date"].min(), "\nEnd:", oil_df["date"].max())
print("Tage insgesamt:", oil_df["date"].nunique())

#### holiday

In [ ]:
holidays_df = pd.read_csv(DATA_DIR / "holidays_events.csv",
                          names=["date", "type", "local", "local-name", "description", "transferred", ],
                          header=0, parse_dates=["date"])

print("=== Head: ===")
print(holidays_df.head())

print("=== Info: ===")
print(holidays_df.info())

print("=== Description: ===")
print(holidays_df.describe())

In [ ]:
# 2. Missing Values
print("=== How much NAs per Col: ===")
print(holidays_df.isnull().sum())

#### Sample submission


In [ ]:
sample_submission = pd.read_csv(DATA_DIR / "sample_submission.csv",
                          names=["id", "sales" ],
                          header=0)

print("=== Head: ===")
print(sample_submission.head())

print("=== Info: ===")
print(sample_submission.info())

print("=== Description: ===")
print(sample_submission.describe())

#### Stores

In [ ]:


print("=== Head: ===")
print(stores.head())

print("=== Info: ===")
print(stores.info())

print("=== Description: ===")
print(stores.describe())

In [ ]:

store_counts = stores['type'].value_counts().sort_index()

plt.figure(figsize=(8,5))
store_counts.plot(kind='bar')
plt.title("Distribution of Stores by Type")
plt.xlabel("Store Type")
plt.ylabel("Number of Stores")
plt.xticks(rotation=0)
plt.show()


In [ ]:


cluster_type_ct = pd.crosstab(stores['cluster'], stores['type'])
plt.figure(figsize=(8,6))
sns.heatmap(cluster_type_ct, annot=True, fmt="d", cmap="Greens")
plt.title("Cluster vs. Store Type")
plt.ylabel("Cluster")
plt.xlabel("Store Type")
plt.show()


In [ ]:


city_cluster_ct = pd.crosstab(stores['city'], stores['cluster'])

values = range(int(city_cluster_ct.values.max()) + 1)
cmap = ListedColormap(sns.color_palette("YlGnBu", len(values)))
norm = BoundaryNorm(values, cmap.N)

plt.figure(figsize=(12,8))
sns.heatmap(
    city_cluster_ct,
    cmap=cmap,
    norm=norm,
    cbar=True,
    linewidths=0.5,
    linecolor="gray",
    annot=True, fmt="d"
)
plt.title("City vs. Cluster Distribution (Discrete)")
plt.ylabel("City")
plt.xlabel("Cluster")
plt.show()


#### transaction


In [ ]:
transaction_df = pd.read_csv(DATA_DIR / "transactions.csv",
                          names=["date", "store_nbr", "transaction" ],
                          header=0, parse_dates=["date"])

print("=== Head: ===")
print(transaction_df.head())

print("=== Info: ===")
print(transaction_df.info())

print("=== Description: ===")
print(transaction_df.describe())

In [ ]:
# count unique store_nbr
transaction_df['store_nbr'].nunique()

In [ ]:
transactions_per_day = transaction_df.groupby('date')['transaction'].sum()


plt.figure(figsize=(12,5))
transactions_per_day.plot()
plt.title("Total Transactions per Day")
plt.xlabel("Date")
plt.ylabel("Transactions")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
transactions_per_day.plot(alpha=0.4, label="Daily")
transactions_per_day.rolling(7).mean().plot(label="7-day Avg")
transactions_per_day.rolling(14).mean().plot(label="14-day Avg")
transactions_per_day.rolling(30).mean().plot(label="30-day Avg")
plt.title("Total Transactions per Day with Rolling Averages")
plt.xlabel("Date")
plt.ylabel("Transactions")
plt.legend()
plt.show()


#### Train/ Test

In [ ]:
train_df = pd.read_csv(DATA_DIR / "train.csv",
                          names=["id", "date", "store_nbr", "family","sales", "onpromotion" ],
                          header=0, parse_dates=["date"])

print("=== Head: ===")
print(train_df.head())

print("=== Info: ===")
print(train_df.info())

print("=== Description: ===")
print(train_df.describe())

In [ ]:
train_df['family'].nunique()

In [ ]:
# missing values
train_df.isna().sum()

In [ ]:

daily_sales = train_df.groupby('date')['sales'].sum()

plt.figure(figsize=(12,5))
ax = daily_sales.plot()

# values into millions
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{float(x/1e6)}M'))

plt.title("Total Sales per Day")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.show()



In [ ]:

weekday_sales = train_df.groupby(train_df['date'].dt.day_name())['sales'].sum()

# define order fpr weekdays
order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
weekday_sales = weekday_sales.reindex(order)

# values into millions
ax = weekday_sales.plot.bar()
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{float(x/1e6)}M'))
ax.xaxis.set_tick_params(rotation=45)


### 1.1.b Identifiy missing data and outliers

#### missing data

In [ ]:
# Missing oil
print("=== How much NAs per Col: ===")
print(oil_df.isnull().sum())

In [ ]:
# Visualization to missing oil data:
import matplotlib.pyplot as plt
s = oil_df.set_index("date")["oil_price"]

fig, ax = plt.subplots(figsize=(12,5))
s.plot(ax=ax)

for x in s.index[s.isna()]:
    ax.axvline(x, linestyle="--", linewidth=0.8, alpha=0.3, color="tab:orange")

ax.set_title("Daily Oil Price")
plt.show()

In [ ]:
tpd = (transaction_df
       .groupby(['store_nbr', 'date'], as_index=False)['transaction']
       .sum()
       .sort_values(['store_nbr', 'date']))
tpd['date'] = pd.to_datetime(tpd['date']).dt.normalize()


full_idx = pd.date_range(tpd['date'].min(), tpd['date'].max(), freq='D')


dates_by_store = tpd.groupby('store_nbr')['date'].unique()
missing_counts = {s: len(full_idx.difference(pd.DatetimeIndex(dates)))
                  for s, dates in dates_by_store.items()}
stores_with_gaps = [s for s, cnt in missing_counts.items() if cnt > 0]

stores_to_plot = sorted(stores_with_gaps, key=lambda s: missing_counts[s], reverse=True)

# Subplots
n = len(stores_to_plot)
cols = 3 if n > 1 else 1
rows = math.ceil(n / cols)

fig, axes = plt.subplots(rows, cols, figsize=(14, 3.2*rows), sharex=True)
axes = axes.flatten() if n > 1 else [axes]

for ax, s in zip(axes, stores_to_plot):
    sdf = (tpd[tpd['store_nbr'] == s]
           .set_index('date')
           .reindex(full_idx))
    ax.plot(sdf.index, sdf['transaction'], linewidth=0.5)

    miss = sdf['transaction'].isna()
    if miss.any():
        ymin, ymax = ax.get_ylim()
        ax.scatter(sdf.index[miss], [ymin]*miss.sum(), marker='v', s=16, alpha=0.7, color="red")
    ax.set_title(f"Store {s}  (missing days: {missing_counts[s]})", fontsize=10)

for ax in axes[n:]:
    ax.axis('off')

fig.tight_layout()
plt.show()


Most stores have 10-12 days missing around Christmas/ New Year, but some stores have a lot more missing data.

In [ ]:
tpd_df = transaction_df.copy()
tpd_df['date'] = pd.to_datetime(tpd_df['date']).dt.normalize()

tpd = (tpd_df
       .groupby(['store_nbr', 'date'], as_index=False)['transaction']
       .sum()
       .sort_values(['store_nbr', 'date']))


stores_to_plot = sorted(tpd['store_nbr'].unique().tolist())


n = len(stores_to_plot)
cols = 4 if n >= 12 else 3 if n >= 6 else 2 if n > 1 else 1
rows = math.ceil(n / cols)

fig, axes = plt.subplots(rows, cols, figsize=(3.2*cols, 3.8*rows))
axes = axes.flatten() if n > 1 else [axes]

for ax, store in zip(axes, stores_to_plot):
    svals = tpd.loc[tpd['store_nbr'] == store, 'transaction']
    sns.boxplot(y=svals, ax=ax, fliersize=2)
    ax.set_title(f"Store {store} (n={svals.notna().sum()})", fontsize=9)
    ax.grid(True, alpha=0.2)

for ax in axes[len(stores_to_plot):]:
    ax.axis('off')

fig.tight_layout()
plt.show()


Looks like there are outliers in every store transaction.

# 2 Data Cleaning
    a. Handling Missing Values
    b. Choose appropriate methods to handle missing values (e.g., mean/median imputation for numerical data, mode imputation for categorical data, or deletion of rows/columns).
    c. Justify your choices for handling missing data.

In [ ]:
oil_df['date'] = pd.to_datetime(oil_df['date'])
oil_df = oil_df.sort_values('date')


oil_df['oil_price'] = oil_df['oil_price'].interpolate(
    method="spline", order=5
)

s = oil_df.set_index("date")["oil_price"]

fig, ax = plt.subplots(figsize=(12,5))
s.plot(ax=ax)


ax.set_title("Daily Oil Price (Spline interpolation)")
plt.show()


In our dataset, only a few short gaps in the oil price data occurred. Since oil prices typically evolve continuously over time without abrupt jumps, interpolation is a suitable approach in this context. We chose spline interpolation because it produces smoother transitions than linear interpolation and thus reflects the economic trajectory more realistically. This allows us to fill the missing values consistently without significantly distorting the statistical properties of the series.

In [ ]:
# handle missing data in store

# implement holiday feature to explain missing data
"""
    is_holiday 1: holiday.type in {holiday, additonal, bridge, Event} && holiday.transfered == false && holiday.locale == National
    is_holiday 1: holiday.type in {holiday, additonal, bridge, Event} && holiday.transfered == false && (holiday.locale == Regional && stores[transactions.str_nbr].state ==  holiday.locale_name)
    is_holiday 1: holiday.type in {holiday, additonal, bridge, Event} && holiday.transfered == false && (holiday.locale == Local && stores[transactions.str_nbr].city ==  holiday.locale_name)

    is_holiday 0: holiday.type == workday || holiday.transfered == true
    is_holiday 0: default with no holiday in the file

    is_holiday 1: holiday.type == Transfered

    => holiday.type == Transfered :: holiday.type to holiday
    => holiday.transfered == true :: holiday.type to workday
"""

transactions = pd.read_csv(DATA_DIR / "transactions.csv",
                          names=["date", "store_nbr", "transaction" ],
                          header=0, parse_dates=["date"])

holidays = pd.read_csv(DATA_DIR / "holidays_events.csv",
                          names=["date", "type", "locale", "locale_name", "description", "transferred", ],
                          header=0, parse_dates=["date"])

stores = pd.read_csv(DATA_DIR / "stores.csv",
                          names=["store_nbr", "city", "state", "type", "cluster" ],
                          header=0)


transactions['date'] = pd.to_datetime(transactions['date']).dt.normalize()
holidays['date'] = pd.to_datetime(holidays['date']).dt.normalize()

# delete an value:
transactions = transactions[transactions['date'] != pd.to_datetime("2013-01-01")]
transactions_grouped = (transactions
                            .groupby(['store_nbr', 'date'], as_index=False)['transaction']
                            .sum()
                            .sort_values(['store_nbr', 'date']))

# holidays preprocessing
holidays_clean = holidays.copy()

# transfers: make them Holiday
holidays_clean.loc[holidays_clean['type'] == "Transfer", 'type'] = "Holiday"

# transferred==True → make them Work Day
holidays_clean.loc[holidays_clean['transferred'] == True, 'type'] = "Work Day"

# join holidays onto transactions
df = transactions_grouped.merge(stores, on="store_nbr", how="left")

# add holiday info
df = df.merge(holidays_clean[['date','type','locale','locale_name','transferred']],
              on="date", how="left",

              suffixes=("", "_holiday"))


# default: no holiday
df['is_holiday'] = 0

# rule 1: national holiday
mask_nat = (df['type_holiday'].isin(["Holiday","Additional","Bridge"])) & (df['locale']=="National")
df.loc[mask_nat, 'is_holiday'] = 1

# rule 2: regional holiday
mask_reg = (df['type_holiday'].isin(["Holiday","Additional","Bridge"])) & (df['locale']=="Regional") & (df['state']==df['locale_name'])
df.loc[mask_reg, 'is_holiday'] = 1

# rule 3: local holiday
mask_loc = (df['type_holiday'].isin(["Holiday","Additional","Bridge"])) & (df['locale']=="Local") & (df['city']==df['locale_name'])
df.loc[mask_loc, 'is_holiday'] = 1

# remove city, state, type, cluster, type_holiday, locale, locale_name, transferred

df = df[['date','store_nbr','transaction','is_holiday']]
print(df.tail(20))


In [ ]:
# Build store open feature
"""
    store_open 0: store is closed if data is missing for more than 10 continuing days
        => add a transaction on dates with 0
    store_open 1: store is open if data is available for at least 10 continuing days
"""
global_start = df['date'].min()
global_end   = df['date'].max()

full_cal = []
for s in df['store_nbr'].unique():
    g = (df.loc[df['store_nbr'] == s, ['date','transaction','is_holiday']]
           .copy()
           .sort_values('date'))

    g = (g.groupby('date', as_index=False)
           .agg(transaction=('transaction','sum'),
                is_holiday =('is_holiday','max')))

    idx = pd.DataFrame({'date': pd.date_range(global_start, global_end, freq='D')})
    g_full = idx.merge(g, on='date', how='left')

    g_full['store_nbr'] = s

    # Holiday-Flag for added days
    g_full['is_holiday'] = g_full['is_holiday'].fillna(0).astype(int)

    full_cal.append(g_full)

df_full = pd.concat(full_cal, ignore_index=True)


def run_mask(series, cond, min_len=10):
    m = cond(series)
    grp = (m != m.shift()).cumsum()
    runlen = m.groupby(grp).transform('size')
    return m & (runlen >= min_len)

df_full = df_full.sort_values(['store_nbr','date'])

open_run = (df_full.groupby('store_nbr')['transaction']
                  .apply(lambda s: run_mask(s, pd.Series.notna, min_len=10))
                  .reset_index(level=0, drop=True))

closed_run = (df_full.groupby('store_nbr')['transaction']
                    .apply(lambda s: run_mask(s, pd.Series.isna, min_len=10))
                    .reset_index(level=0, drop=True))
df_full['store_open'] = np.where(open_run, 1,
                          np.where(closed_run, 0, np.nan))

df_full['store_open'] = (
    df_full
      .groupby('store_nbr', group_keys=False)['store_open']
      .apply(lambda s: s.ffill().bfill())
      .fillna(0)
      .astype(int)
)

df_full.loc[(df_full['store_open']==0) & (df_full['transaction'].isna()), 'transaction'] = 0

def interp_per_store(g, max_gap=5):
    g = g.sort_values('date').set_index('date')
    allowed = (g['store_open'] == 1) & (g['is_holiday'] == 0)
    y = g['transaction'].copy()
    # only interpolate allowed data
    y_allowed = y.where(allowed)
    y_filled = y_allowed.interpolate(method='time',
                                     limit=max_gap,
                                     limit_area='inside')
    g.loc[allowed & y.isna(), 'transaction'] = y_filled
    return g.reset_index()

df_full = df_full.groupby('store_nbr', group_keys=False).apply(interp_per_store, max_gap=5)

print(df_full.tail(20))

### Data Cleaning – Transactions

- A single value (2013-01-01) was removed, since data was only available for one store.
- Holiday data was integrated and marked with `is_holiday` to explain transaction peaks.
- A complete daily calendar was built for each store and `store_open` was derived:
  - `0` if ≥10 consecutive days without data (closed, transactions = 0).
  - `1` if ≥10 consecutive days with data (open).
- Missing values in open, non-holiday phases were only interpolated for short gaps (≤5 days) using time-based interpolation.

➡️ This way, real closures and events remain intact, while small gaps are handled consistently.


In [ ]:
# ===================================================
#                   VISUALIZATION
# ===================================================
stores_unique = df_full['store_nbr'].unique()

for store in stores_unique:
    store_df = df_full[df_full['store_nbr'] == store].sort_values('date')

    fig, ax = plt.subplots(figsize=(16,5))
    ax.plot(store_df['date'], store_df['transaction'], label="Transactions", color="blue", linewidth=0.4, alpha=1)



    """holiday_dates = store_df.loc[store_df['is_holiday'] == 1, 'date']
    for hday in holiday_dates:
        ax.axvline(hday, color="orange", linestyle="--", linewidth=0.5)
    """
    """
    closed_dates = store_df.loc[store_df['store_open'] == 0, 'date']
    for cdate in closed_dates:
        ax.axvline(cdate, color="orange", linestyle="-", linewidth=0.5)
    """
    open_but_no_data = store_df.loc[
        (store_df['store_open'] == 1) &
        (store_df['is_holiday'] == 0) &
        (store_df['transaction'].isna()),
        'date'
    ]
    for obnd in open_but_no_data:
        ax.axvline(obnd, color="red", linestyle="-", linewidth=0.5)

    ax.set_title(f"Store {store} - Transactions with Holidays")
    ax.set_xlabel("Date")
    ax.set_ylabel("Transactions")
    ax.legend()

    plt.show()

# 3 Handling Outliers
    a. Detect outliers using methods such as the IQR method or Z-score.
    b. Decide whether to remove, cap, or transform the outliers. Justify your decisions.

In [ ]:
# 1) Load data
train = pd.read_csv("train.csv", parse_dates=["date"])
transactions = pd.read_csv("transactions.csv", parse_dates=["date"])
oil = pd.read_csv("oil.csv", parse_dates=["date"])

# 2) Merge data
df = train.merge(transactions, on=["date", "store_nbr"], how="left")
df = df.merge(oil, on="date", how="left")

# Numeric columns of interest
num_cols = ["sales", "onpromotion", "transactions", "dcoilwtico"]

# 3) Basic cleaning
for col in ["sales", "onpromotion", "transactions"]:
    # Negative values are not valid → set to NaN
    df.loc[df[col] < 0, col] = np.nan

# Interpolate missing oil prices
oil_sorted = oil.sort_values("date").set_index("date")
oil_sorted["dcoilwtico"] = oil_sorted["dcoilwtico"].interpolate(method="time")
df = df.drop(columns=["dcoilwtico"]).merge(oil_sorted.reset_index(), on="date", how="left")

# 4) Outlier detection (IQR and Z-Score)
iqr_bounds, iqr_counts, z_counts = {}, {}, {}

for col in num_cols:
    series = df[col].dropna()
    q1, q3 = series.quantile(0.25), series.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    iqr_bounds[col] = (lower, upper)
    iqr_counts[col] = int(((df[col] < lower) | (df[col] > upper)).sum())

    z = np.abs(stats.zscore(series, nan_policy="omit"))
    z_counts[col] = int((z > 3).sum())

print("Outlier counts (IQR):", iqr_counts)
print("Outlier counts (Z-Score):", z_counts)

# 5) Visual inspection
def plot_hist(col, bins=60):
    plt.figure(figsize=(7,4))
    plt.hist(df[col].dropna(), bins=bins)
    plt.title(f"Histogram of {col}")
    plt.show()

def plot_box(col):
    plt.figure(figsize=(7,2.5))
    plt.boxplot(df[col].dropna(), vert=False, whis=1.5)
    plt.title(f"Boxplot of {col}")
    plt.show()

for c in num_cols:
    plot_hist(c)
    plot_box(c)

# 6) Outlier handling strategy
df_handled = df.copy()

# sales: keep raw, add log1p transformation
df_handled["sales_log1p"] = np.log1p(df_handled["sales"])

# onpromotion: cap at 99.5th percentile
p995_onpromo = df_handled["onpromotion"].quantile(0.995)
df_handled["onpromotion_capped"] = np.where(
    df_handled["onpromotion"] > p995_onpromo, p995_onpromo, df_handled["onpromotion"]
)

# transactions: cap at 99.5th percentile
p995_trans = df_handled["transactions"].quantile(0.995)
df_handled["transactions_capped"] = np.where(
    df_handled["transactions"] > p995_trans, p995_trans, df_handled["transactions"]
)

# oil price: winsorize between 1st and 99th percentiles
p01_oil, p99_oil = df_handled["dcoilwtico"].quantile([0.01, 0.99])
df_handled["dcoilwtico_winsor"] = df_handled["dcoilwtico"].clip(lower=p01_oil, upper=p99_oil)

print("Caps applied -> onpromotion 99.5%:", p995_onpromo,
      "| transactions 99.5%:", p995_trans,
      "| oil [1%, 99%]:", (p01_oil, p99_oil))

# 7) Save cleaned dataset
df_handled.to_csv("train_outlier_handled.csv", index=False)


# 4 Data Transformation
    a. Encoding Categorical Data
        i. Apply label encoding or one-hot encoding to transform categorical data into numerical form.
        ii. Justify your choice of encoding method.
    b. Feature Scaling
        i. Apply feature scaling techniques such as normalization (Min-Max scaling) or standardization (Z-score normalization) to the dataset.
        ii. Explain why feature scaling is necessary and how it impacts the model.


# 5 Data Splitting
    a. Split the preprocessed dataset into training and testing sets. Typically, an 80-20 or 70-30 split is used.


In [9]:
import pandas as pd

# Parameters
train_fraction = 0.8

# Load datasets
train_df = pd.read_csv("train.csv", parse_dates=["date"])
test_df = pd.read_csv("test.csv", parse_dates=["date"])

# Combine ALL datasets (test.csv will have NaN in sales column)
all_cols = sorted(set(train_df.columns).union(set(test_df.columns)))
train_df = train_df.reindex(columns=all_cols)
test_df = test_df.reindex(columns=all_cols)
full_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

# Sort by date chronologically
full_df = full_df.sort_values("date").reset_index(drop=True)

# Chronological split on ALL data: first 80% by time for training
n = len(full_df)  # This is 3,029,400
split_idx = int(n * train_fraction)  # This is 2,423,520

train_part = full_df.iloc[:split_idx]
test_part = full_df.iloc[split_idx:]

# Separate features and target (some sales values will be NaN)
X_train = train_part.drop("sales", axis=1)
y_train = train_part["sales"]
X_test = test_part.drop("sales", axis=1)
y_test = test_part["sales"]

# Results
print("Total data:", n)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print("NaN values in y_train:", y_train.isna().sum())
print("NaN values in y_test:", y_test.isna().sum())


Total data: 3029400
X_train shape: (2423520, 5)
X_test shape: (605880, 5)
y_train shape: (2423520,)
y_test shape: (605880,)
NaN values in y_train: 0
NaN values in y_test: 28512


    b. Explain the importance of splitting the data and how it prevents overfitting.

We split the dataset into training (80%) and testing (20%) sets. The training set is used to teach the model, while the testing set evaluates its performance on unseen data. This prevents overfitting, ensuring the model generalizes well rather than just memorizing the training data.

# 6 Bonus
Apply dimensionality reduction techniques such as Principal
Component Analysis (PCA) and discuss how it affects the dataset.